---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
#import matplotlib.pyplot as plt
#%matplotlib inline


---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))


<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [4]:
def graph_identification():
    i=0
    algo=[]
    type_algorithm=[]
    
    for G in P1_Graphs:
        degrees = G.degree()

        degree_values = sorted(set(degrees.values()))

        histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]

        avg_clustering=nx.average_clustering(G)
        avg_shortest_path=nx.average_shortest_path_length(G)
        
        #print(nx.info(G)) 
        
        #plt.bar(degree_values,histogram) 
        #plt.xlabel('Degree') 
        #plt.ylabel('Fraction of Nodes') 
        #plt.show()
        #algo.append((len(histogram),avg_clustering,avg_shortest_path))
        
        if len(histogram) >15:
            type_algorithm.append('PA')
        elif avg_clustering > 0.1:
            type_algorithm.append('SW_L')
        else:
            type_algorithm.append('SW_H')
        
    return type_algorithm

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')



### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

def salary_predictions():
    
    df=pd.DataFrame(G.nodes())
    df['Department'] =pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary']=pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['BtwnCent']  = pd.Series(nx.betweenness_centrality(G, normalized = True, endpoints = False))
    df['Degree']=pd.Series(G.degree())
    df['closeCent']  = pd.Series(nx.closeness_centrality(G))
    df['DegCentrality']=pd.Series(nx.degree_centrality(G))
    df['Clustering']=pd.Series(nx.clustering(G))
    df['PageRank'] = pd.Series(nx.pagerank(G))
 
    df_train = df[pd.notnull(df['ManagementSalary'])] 
    df_test = df[pd.isnull(df['ManagementSalary'])] 
    
    y_train= df_train['ManagementSalary']
    y_test=df_test['ManagementSalary']
    
    X_train=df_train.drop(['ManagementSalary'],axis=1)
    X_test=df_test.drop(['ManagementSalary'],axis=1)
    
    scaler=MinMaxScaler()
    
    X_train_scaler=scaler.fit_transform(X_train)
    X_test_scaler=scaler.fit_transform(X_test)
    
    
    clf_RandomForest=RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    clf_RandomForest.fit(X_train_scaler, y_train)
    
    y_predict= clf_RandomForest.predict_proba(X_test_scaler)[:,1].tolist()

    return pd.Series(y_predict,X_test.index)

salary_predictions()

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [14]:
def new_connections_predictions():
    
    future_connections['Common Neighbor']=[len(list(nx.common_neighbors(G, e[0],e[1]))) for e in future_connections.index]
    
    future_connections['Jacard Coeff']=[x[2] for x in list(nx.jaccard_coefficient(G))]
    
    
    future_connections['Resource Alloc'] = [x[2] for x in list(nx.resource_allocation_index(G))]

    future_connections['AdamicAdar Index']=[x[2] for x in list(nx.adamic_adar_index(G))]
        
    future_connections['Pref Attachment']=[x[2] for x in list(nx.preferential_attachment(G))]
    
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']

    future_connections['Community']=[x[2] for x in list(nx.cn_soundarajan_hopcroft(G))]
    future_connections['Community']=[x[2] for x in list(nx.ra_index_soundarajan_hopcroft(G))]
    
    df_train = future_connections[pd.notnull(future_connections['Future Connection'])] 
    df_test = future_connections[pd.isnull(future_connections['Future Connection'])] 
    
    y_train= df_train['Future Connection']
    y_test=df_test['Future Connection']
    
    X_train=df_train.drop(['Future Connection'],axis=1)
    X_test=df_test.drop(['Future Connection'],axis=1)
    
    scaler=MinMaxScaler()
    X_train_scaler=scaler.fit_transform(X_train)
    X_test_scaler=scaler.fit_transform(X_test)
    
    
    clf_RandomForest=RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    clf_RandomForest.fit(X_train_scaler, y_train)
    
    y_predict= clf_RandomForest.predict_proba(X_test_scaler)[:,1].tolist()

    return pd.Series(y_predict,X_test.index)

new_connections_predictions()

(107, 348)    0.057963
(542, 751)    0.057506
(20, 426)     0.420285
(50, 989)     0.058930
(942, 986)    0.058451
(324, 857)    0.058972
(13, 710)     0.226597
(19, 271)     0.225266
(319, 878)    0.058930
(659, 707)    0.058930
(49, 843)     0.057506
(208, 893)    0.057506
(377, 469)    0.057963
(405, 999)    0.058972
(129, 740)    0.058696
(292, 618)    0.057506
(239, 689)    0.058972
(359, 373)    0.058696
(53, 523)     0.057914
(276, 984)    0.057963
(202, 997)    0.057963
(604, 619)    0.058930
(270, 911)    0.057963
(261, 481)    0.080174
(200, 450)    0.420188
(213, 634)    0.058679
(644, 735)    0.057914
(346, 553)    0.058050
(521, 738)    0.058972
(422, 953)    0.056369
                ...   
(672, 848)    0.058077
(28, 127)     0.390524
(202, 661)    0.059062
(54, 195)     0.389464
(295, 864)    0.059322
(814, 936)    0.058930
(839, 874)    0.059062
(139, 843)    0.059062
(461, 544)    0.059062
(68, 487)     0.058077
(622, 932)    0.059321
(504, 936)    0.059358
(479, 528) 